# 🔄 Transition Dynamics Analysis: Hysteresis Loop

**Мета**: Дослідження динаміки переходів між станами (Normal ↔ Attack)

## Ключові питання:
1. Як швидко відбувається **деградація** сигналу при атаці?
2. Як швидко відбувається **відновлення** після атаки?
3. Чи існує **гістерезис** (асиметрія швидкостей)?

## Фізичний зміст:
- **Onset (0→1)**: Момент початку атаки
- **Recovery (1→0)**: Момент завершення атаки
- **Hysteresis**: Різниця в динаміці падіння та відновлення

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
print("✅ Бібліотеки імпортовано")

## 1. Завантаження та підготовка даних

In [ ]:
DATA_PATH = '../data/processed/all_data_compressed.parquet'
df = pd.read_parquet(DATA_PATH)
df = df.sort_values('timestamp').reset_index(drop=True)

# Конвертація часу
if not np.issubdtype(df['timestamp'].dtype, np.datetime64):
    df['timestamp'] = pd.to_datetime(df['timestamp'])

# Feature engineering
if 'numSV' in df.columns and 'numSatsTracked' in df.columns:
    df['sat_efficiency'] = df['numSV'] / df['numSatsTracked']
    df['sat_efficiency'] = df['sat_efficiency'].replace([np.inf, -np.inf], np.nan).fillna(0)
else:
    df['sat_efficiency'] = 0

print(f"📊 Завантажено: {len(df):,} записів")
print(f"🕐 Період: {df['timestamp'].min()} — {df['timestamp'].max()}")

## 2. Виявлення моментів переходу

In [ ]:
label = df['overallPositionLabel'].values

# Onset: 0 → 1 (початок атаки)
onset_idx = np.where((label[:-1] == 0) & (label[1:] == 1))[0] + 1

# Recovery: 1 → 0 (кінець атаки)
recovery_idx = np.where((label[:-1] == 1) & (label[1:] == 0))[0] + 1

print(f"🔴 Onset events (0→1): {len(onset_idx):,}")
print(f"🟢 Recovery events (1→0): {len(recovery_idx):,}")

## 3. Витягування вікон навколо переходів

In [ ]:
window_size = 30  # секунд до і після переходу

def extract_windows(indices, window_size):
    """Витягує вікна метрик навколо кожного переходу"""
    windows = []
    for idx in indices:
        if idx - window_size >= 0 and idx + window_size < len(df):
            win = df.iloc[idx-window_size:idx+window_size+1][['sat_efficiency', 'cnoMean']].copy()
            win = win.reset_index(drop=True)
            win['rel_time'] = np.arange(-window_size, window_size+1)
            windows.append(win)
    return windows

windows_onset = extract_windows(onset_idx, window_size)
windows_recovery = extract_windows(recovery_idx, window_size)

print(f"📊 Зібрано вікон:")
print(f"   Onset: {len(windows_onset)}")
print(f"   Recovery: {len(windows_recovery)}")

## 4. Візуалізація: Hysteresis Loop

Графік показує динаміку зміни **Satellite Efficiency** при переході в атаку (ліворуч) та виході з атаки (праворуч).

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharey=True)

# --- Onset (0→1) ---
for win in windows_onset:
    axes[0].plot(win['rel_time'], win['sat_efficiency'], 
                 color='#3498db', alpha=0.05, linewidth=1)

if windows_onset:
    mean_onset = pd.concat(windows_onset).groupby('rel_time')['sat_efficiency'].mean()
    axes[0].plot(mean_onset.index, mean_onset.values, 
                 color='#2980b9', linewidth=3, label='Mean Trajectory')

axes[0].axvline(x=0, color='red', linestyle='--', linewidth=2, label='Transition Point')
axes[0].fill_betweenx([0, 1], -5, 0, alpha=0.1, color='green', label='Pre-attack Zone')
axes[0].fill_betweenx([0, 1], 0, 5, alpha=0.1, color='red', label='Attack Zone')
axes[0].set_title('🔴 Attack ONSET (Normal → Attack)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Time (seconds from transition)', fontsize=12)
axes[0].set_ylabel('Satellite Efficiency', fontsize=12)
axes[0].legend(loc='lower left')
axes[0].set_xlim(-window_size, window_size)

# --- Recovery (1→0) ---
for win in windows_recovery:
    axes[1].plot(win['rel_time'], win['sat_efficiency'], 
                 color='#27ae60', alpha=0.05, linewidth=1)

if windows_recovery:
    mean_recovery = pd.concat(windows_recovery).groupby('rel_time')['sat_efficiency'].mean()
    axes[1].plot(mean_recovery.index, mean_recovery.values, 
                 color='#1e8449', linewidth=3, label='Mean Trajectory')

axes[1].axvline(x=0, color='red', linestyle='--', linewidth=2, label='Transition Point')
axes[1].fill_betweenx([0, 1], -5, 0, alpha=0.1, color='red', label='Attack Zone')
axes[1].fill_betweenx([0, 1], 0, 5, alpha=0.1, color='green', label='Recovery Zone')
axes[1].set_title('🟢 RECOVERY (Attack → Normal)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Time (seconds from transition)', fontsize=12)
axes[1].legend(loc='lower right')
axes[1].set_xlim(-window_size, window_size)

plt.suptitle('HYSTERESIS LOOP: Asymmetry in Attack Onset vs Recovery Dynamics', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('figures/hysteresis_loop.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Кількісний аналіз: швидкість переходу

In [ ]:
def calculate_transition_slope(windows, metric, window_half=5):
    """
    Розраховує середній нахил (швидкість зміни) метрики
    в околі точки переходу (t=0)
    """
    slopes = []
    for win in windows:
        y = win[metric].values
        center = window_size  # index at t=0
        # Slope from t=-window_half to t=0
        if center - window_half >= 0 and center < len(y):
            slope = (y[center] - y[center - window_half]) / window_half
            slopes.append(slope)
    return np.mean(slopes), np.std(slopes), len(slopes)

# Sat Efficiency
onset_slope, onset_std, onset_n = calculate_transition_slope(windows_onset, 'sat_efficiency')
recovery_slope, recovery_std, recovery_n = calculate_transition_slope(windows_recovery, 'sat_efficiency')

# CNO Mean
onset_slope_cno, onset_std_cno, _ = calculate_transition_slope(windows_onset, 'cnoMean')
recovery_slope_cno, recovery_std_cno, _ = calculate_transition_slope(windows_recovery, 'cnoMean')

print("="*60)
print("📊 TRANSITION DYNAMICS ANALYSIS")
print("="*60)
print(f"\n🔷 Satellite Efficiency (5s slope):")
print(f"   Onset:    {onset_slope:+.4f} ± {onset_std:.4f} (n={onset_n})")
print(f"   Recovery: {recovery_slope:+.4f} ± {recovery_std:.4f} (n={recovery_n})")

print(f"\n🔷 CNO Mean (5s slope):")
print(f"   Onset:    {onset_slope_cno:+.4f} ± {onset_std_cno:.4f}")
print(f"   Recovery: {recovery_slope_cno:+.4f} ± {recovery_std_cno:.4f}")

In [ ]:
# Інтерпретація результату
print("\n" + "="*60)
print("🔬 ІНТЕРПРЕТАЦІЯ:")
print("="*60)

asymmetry = abs(recovery_slope) / abs(onset_slope) if onset_slope != 0 else 0

if abs(recovery_slope) > abs(onset_slope):
    print(f"✅ Recovery ШВИДШЕ за Failure!")
    print(f"   Коефіцієнт асиметрії: {asymmetry:.2f}x")
    print(f"   Система швидко відновлюється після атаки.")
elif abs(recovery_slope) < abs(onset_slope):
    print(f"⚠️ Failure ШВИДШЕ за Recovery!")
    print(f"   Коефіцієнт асиметрії: {1/asymmetry:.2f}x")
    print(f"   Система повільно відновлюється — це проблема!")
else:
    print(f"➡️ Failure та Recovery мають однакову швидкість.")

## 6. Візуалізація асиметрії

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

metrics = ['Sat Efficiency', 'CNO Mean']
onset_slopes = [abs(onset_slope), abs(onset_slope_cno)]
recovery_slopes = [abs(recovery_slope), abs(recovery_slope_cno)]

x = np.arange(len(metrics))
width = 0.35

bars1 = ax.bar(x - width/2, onset_slopes, width, label='Onset (Failure)', color='#e74c3c')
bars2 = ax.bar(x + width/2, recovery_slopes, width, label='Recovery', color='#27ae60')

ax.set_ylabel('|Slope| (rate of change)', fontsize=12)
ax.set_title('Transition Speed Comparison: Onset vs Recovery', fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.legend()
ax.grid(axis='y', alpha=0.3)

# Додаємо значення на барах
for bar in bars1 + bars2:
    height = bar.get_height()
    ax.annotate(f'{height:.4f}',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3), textcoords="offset points",
                ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.savefig('figures/transition_asymmetry.png', dpi=150, bbox_inches='tight')
plt.show()

## 📌 Висновки

### Hysteresis Effect:
- **Гістерезис** — це асиметрія між швидкістю деградації та відновлення
- Виявлена асиметрія важлива для калібрування порогів детектора

### Практичне застосування:
1. **Якщо Recovery > Onset**: можна використовувати агресивніші пороги
2. **Якщо Onset > Recovery**: потрібні консервативніші пороги та holdoff-час
3. **Для Early Warning**: фокус на pre-onset zone (-5s до 0)

### Рекомендації для моделі:
- Використовувати **rolling features** для захоплення динаміки
- Включити **похідні** (rate of change) як ознаки
- Враховувати **holdoff period** після тривоги